In [1]:
!git clone https://github.com/hoangngoclam298/Siamese-Triplet-Model.git

Cloning into 'Siamese-Triplet-Model'...
remote: Enumerating objects: 86, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 86 (delta 28), reused 71 (delta 13), pack-reused 0
Unpacking objects: 100% (86/86), 4.05 MiB | 12.17 MiB/s, done.


In [2]:
%cd /kaggle/working/Siamese-Triplet-Model

/kaggle/working/Siamese-Triplet-Model


In [3]:
from torchvision import datasets, transforms
import numpy as np
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F
import os
from torch.utils.data import Dataset, DataLoader
import random
import torch
import shutil
import time
from torch.optim import lr_scheduler
import torch.optim as optim
from networks import *
from datasets import *
from losses import *
from metrics import *
from trainer import *
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

In [4]:
import os

path_root = '/kaggle/input/data_train_2d_new/data_train_2d'
new_path_train = '/kaggle/working/data_train'
new_path_test = '/kaggle/working/data_test'

if os.path.exists(new_path_train):
    shutil.rmtree(new_path_train)
os.makedirs(new_path_train, exist_ok=True)

if os.path.exists(new_path_test):
    shutil.rmtree(new_path_test)
os.makedirs(new_path_test, exist_ok=True)

list_id = os.listdir(path_root)
random.shuffle(list_id)
list_train = ['1038', '1190', '1187', '1041', '2175', '1134', '2059', '1220', '1119', '2198', '2005', '1031', '1008', '1042', '1221', '1080', '2170', '1032', '2029', '2046', '1075', '2017', '1012', '2213', '2023', '2002', '2072', '2197', '1089', '2183', '1027', '1189', '2215', '2200', '2191', '2214', '1186', '2193', '1182', '1002', '2223', '1050', '1188', '1133', '1200', '2075', '1151', '1116', '2006', '2207', '1022', '1208', '1024', '1199', '1088', '2177', '2009', '2185', '1195', '1213', '2051', '2021', '1099', '2052', '2019', '2218', '2205', '2226', '1013', '2173', '1003', '1144', '1175', '1085', '1196', '2209', '1074', '2068', '2188', '2195', '2086', '2016', '1039', '1217', '2011', '2186', '1037', '1181', '1173', '2203', '1066', '1004', '2180', '2008', '1201', '1197', '1192', '2201', '1174', '1184', '2060', '2064', '2181', '2012']
list_test = ['2172', '1040', '2189', '2174', '2184', '1043', '2219', '2179', '2225', '1072', '2210', '2074', '1001', '2001', '1014', '2176', '1177', '2141', '2004', '1067', '2211', '2054', '2178', '1047', '2190', '1025', '1009']

for id in list_train:
    path_id = os.path.join(path_root, id)
    list_id_image = [x for x in os.listdir(path_id) if x.endswith('.png')]
    if len(list_id_image) > 1:
        if len(list_id_image) > 5:
            list_id_image = random.sample(list_id_image, 5)
        os.makedirs(os.path.join(new_path_train, id), exist_ok=True)
        for id_image in list_id_image:
            source_file = os.path.join(path_id, id_image)
            destination_file = os.path.join(new_path_train, id, id_image)
            shutil.copy(source_file, destination_file)

for id in list_test:
    path_id = os.path.join(path_root, id)
    list_id_image = [x for x in os.listdir(path_id) if x.endswith('.png')]
    if len(list_id_image) > 1:
        if len(list_id_image) > 5:
            print(id)
            list_id_image = random.sample(list_id_image, 5)
        os.makedirs(os.path.join(new_path_test, id), exist_ok=True)
        for id_image in list_id_image:
            source_file = os.path.join(path_id, id_image)
            destination_file = os.path.join(new_path_test, id, id_image)
            shutil.copy(source_file, destination_file)

In [5]:
cuda = torch.cuda.is_available()

trans_train = transforms.Compose([
#     transforms.RandomRotation(degrees=(-15, 15)),
    RandomResizedCropRect(size=(256, 256)),
    GaussianNoise(),
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization
])

trans_val = transforms.Compose([
#     transforms.RandomRotation(degrees=(-15, 15)),
    transforms.Resize((256, 256)),
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization
])

train_dataset = TripletDataset(data_dir=new_path_train, transform=trans_train, train=True)
test_dataset = TripletDataset(data_dir=new_path_test, transform=trans_val, train=False)

batch_size = 8
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
triplet_train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, **kwargs)
triplet_test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, **kwargs)

workers = 0 if os.name == 'nt' else 8

train_dataset_image = datasets.ImageFolder('/kaggle/working/data_train', transform=trans_train)
train_loader_image = DataLoader(
    train_dataset_image,
    num_workers=workers,
    batch_size=batch_size,
)

val_dataset_image = datasets.ImageFolder('/kaggle/working/data_test', transform=trans_val)
val_loader_image = DataLoader(
    val_dataset_image,
    num_workers=workers,
    batch_size=batch_size,
)

In [6]:
margin = 1.
embedding_net = EmbeddingNet(len_embedding=512)
model = TripletNet(embedding_net)

if cuda:
    model.cuda()
loss_fn = TripletLoss(margin)
lr = 1e-4
optimizer = optim.Adam(model.parameters(), lr=lr)
# scheduler = lr_scheduler.StepLR(optimizer, 8, gamma=0.1, last_epoch=-1)
scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=50, T_mult=1, eta_min=1e-6)

n_epochs = 2000
log_interval = 1
ts = time.time()
fit(triplet_train_loader, triplet_test_loader, model, loss_fn, optimizer, scheduler, n_epochs, cuda, log_interval, save_best_loss=True, path_save='/kaggle/working/best_model.pt', num_epoch_roc=10, train_loader_image=train_loader_image, val_loader_image=val_loader_image)
print(time.time()-ts)
torch.save(model.state_dict(), '/kaggle/working/last_model.pt')

Epoch: 1/2000. Train set: Average loss: 26.9412
Epoch: 1/2000. Validation set: Average loss: 5.6383
Epoch: 2/2000. Train set: Average loss: 17.3454
Epoch: 2/2000. Validation set: Average loss: 2.3287
Epoch: 3/2000. Train set: Average loss: 12.6529
Epoch: 3/2000. Validation set: Average loss: 2.9899
Epoch: 4/2000. Train set: Average loss: 12.7619
Epoch: 4/2000. Validation set: Average loss: 2.4574
Epoch: 5/2000. Train set: Average loss: 8.4613
Epoch: 5/2000. Validation set: Average loss: 2.8920
Epoch: 6/2000. Train set: Average loss: 7.6388
Epoch: 6/2000. Validation set: Average loss: 2.5959
Epoch: 7/2000. Train set: Average loss: 7.8919
Epoch: 7/2000. Validation set: Average loss: 2.4467
Epoch: 8/2000. Train set: Average loss: 7.8275
Epoch: 8/2000. Validation set: Average loss: 1.9784
Epoch: 9/2000. Train set: Average loss: 6.3226
Epoch: 9/2000. Validation set: Average loss: 1.4404
---Epoch: 9/2000. Train set: Distance ROC: 0.7877	Cosine similarity ROC: 0.8027
---Epoch: 9/2000. Validat